# 数据处理

## 数据处理 - 自来水


>     col 0	5位数字ID，前两位代表“县”
>     col 1	多数情况代表地点，少数情况代表企业
>     col 5	水量
>     col 9	硝酸盐浓度
>     col 16	Na浓度
>     col 18	Cl浓度
>     col 19	硬度
>     col 20	蒸发残留物
>     col 22	TOC
> 
> 以上数据，需要对同一个ID（col 0），以水量（col 5）为权重，加权平均  
> 然后对于同一个县（col 0的前两位）所有的ID，以该地点（col 1）的人口为权重，加权平均


In [96]:
import numpy as np
import pandas as pd
pd.set_option('display.width', 200)   # 每行最大字符
pd.set_option('precision', 3)         # 显示数字精度

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.transforms as transforms

plt.rcParams['figure.figsize'] = 18, 9
plt.rcParams['axes.unicode_minus'] = False     # 显示数字负号
plt.rcParams['font.sans-serif'] = ['SimHei']   # 显示中文字体
mpl.rcParams['figure.dpi'] = 80
mpl.rcParams['savefig.dpi'] = 100
mpl.rcParams['font.size'] = 12
mpl.rcParams['legend.fontsize'] = 'large'
mpl.rcParams['figure.titlesize'] = 'medium'
plt.style.use('seaborn-whitegrid')

import seaborn as sns


### 预览

数据分为两部分

water_map_jp 180917.xlsx:

- 数据量大
- "地点or企业" 可能是 "市/町", 也可能是 "水道企業団" 或其他

Enterprise to location.xlsx:

- 数据量小
- 可以通过企业ID关联到 water_map_jp 180917.xlsx
- 也包含测量结果, 不使用


2019-08-06 

> 更合理的处理方式是，在water_df里，先行把企业都替换成地点，然后再对同一地点进行“水量”加权平均。  
> 举个例子，water第56行，01-117这个企业，对映enterprise第234-235行。这个公司给col(D)的三个地方供水。
> 那么输出是三行：北海道xx1町，北海道xx2町，北海道xx3町。
> 这三行的数值都一样，都是这个公司两家水厂的加权平均。

先前做法是直接在 "water_map_jp 180917.xlsx" 里, 以 "地点or企业" 聚合, 以水量作为权重计算平均

现在应该从中消除企业:

1. 计算公司的加权平均 (可以直接用之前的数据, 已经包含了 地点+公司 的加权平均数据)
2. 统计 water 中的 "地点or企业", 检出所有不是 "市/町" 的值, 按照 ID 查找在 enterprise 表中对应了哪几个地点
3. 将该 "公司名" 的行拆分为 "多个地点" 的行, 测量数据使用先前的公司加权平均
4. 此时地点全为 "市/町", (同一个市/町可能具有多条不同数据) 按照地点再次加权平均, 权重为水量



In [97]:
water_df = pd.read_excel("../water_map_jp 180917.xlsx", sheet_name="raw")

In [98]:
print('预览前几行')
water_df.head(5)

预览前几行


,事業主体ID,事業主体名,浄水場名,水源名,原水の種類,1日平均浄水量,Unnamed: 6,大腸菌(定量)(MPN/100ml),大腸菌(定性),硝酸態窒素及び亜硝酸態窒素,...,銅及びその化合物,ナトリウム及びその化合物,マンガン及びその化合物,塩化物イオン,カルシウム、マグネシウム等(硬度),蒸発残留物,陰イオン界面活性剤,有機物(TOCの量),色度,濁度
0,01-005,北海道稚内市,01-00萩ヶ丘浄水場,声問川水系北辰ダム,ダム直接,18859,10,0,0,0.50,...,<0.01,11.3,0.016,14.5,16,70,<0.02,1.5,10.3,1.7
1,01-005,北海道稚内市,02-00沼川第1浄水場,声問川水系北辰ダム,ダム直接,221,10,0,0,0.50,...,<0.01,11.3,0.016,14.5,16,70,<0.02,1.5,10.3,1.7
2,01-005,北海道稚内市,03-00沼川第2浄水場,声問川水系北辰ダム,ダム直接,810,10,0,0,0.50,...,<0.01,11.3,0.016,14.5,16,70,<0.02,1.5,10.3,1.7
3,01-010,北海道浦河町,01-00野深浄水場,1号井戸、2号井戸,浅井戸水,3800,1,0,0,0.36,...,0,3.7,<0.001,3.5,29,60,<0.02,0.3,<1.0,<0.1
4,01-015,北海道木古内町,01-00木古内浄水場,木古内川水系支流中野川,伏流水,2010,13,0,0,0.17,...,0.02,10,<0.001,12.7,21,70,<0.02,0.5,2,0.3


In [99]:
def explore_df(df):
  print('==== row, col ====', df.shape)
  print('==== columns ====')
  print('\t'.join(df.columns))

  print('==== describe ====')
  print(df.describe())
  print('==== info ====')
  print(df.info())
  print('==== sample ====')
  return df.sample(n=5, random_state=100)
  
# explore_df(df)

def find_in_df(df, col, pat):
  ''' 在 col 中 查找命中 pat 的数据
      如果 pat 是 字符串, 视为模糊查找 
      如果 pat 是 list, 或者非字符串型, 视为精确查找 '''
  if isinstance(pat, str):
    return df[df[col].str.contains(pat)]
  if not isinstance(pat, (list, set)):
    pat = [pat]
  return df[df[col].isin(pat)]


In [100]:
# explore_df(water_df)

In [101]:
print('行, 列', water_df.shape)
print(water_df.columns)

行, 列 (3619, 25)
Index(['事業主体ID', '事業主体名', '浄水場名', '水源名', '原水の種類', '1日平均浄水量', 'Unnamed: 6', '大腸菌(定量)(MPN/100ml)', '大腸菌(定性)', '硝酸態窒素及び亜硝酸態窒素', 'フッ素及びその化合物', 'ホウ素及びその化合物', '四塩化炭素', 'アルミニウム及びその化合物', '鉄及びその化合物',
       '銅及びその化合物', 'ナトリウム及びその化合物', 'マンガン及びその化合物', '塩化物イオン', 'カルシウム、マグネシウム等(硬度)', '蒸発残留物', '陰イオン界面活性剤', '有機物(TOCの量)', '色度', '濁度'],
      dtype='object')


### 保留有用的column, 重命名column

In [102]:
column_map = [
  ('事業主体ID',                       '5位数字ID'),
  ('事業主体名',                       '地点或企业'),
  ('1日平均浄水量',                    '水量'),
  ('硝酸態窒素及び亜硝酸態窒素',        '硝酸盐浓度'),
  ('ナトリウム及びその化合物',          'Na浓度'),
  ('塩化物イオン',                     'Cl浓度'),
  ('カルシウム、マグネシウム等(硬度)',   '硬度'),
  ('蒸発残留物',                       '蒸发残留物'),
  ('有機物(TOCの量)',                  'TOC'),
]

water_df = water_df[[line[0] for line in column_map]]
water_df.columns = [line[1] for line in column_map]
water_df.head(5)

,5位数字ID,地点或企业,水量,硝酸盐浓度,Na浓度,Cl浓度,硬度,蒸发残留物,TOC
0,01-005,北海道稚内市,18859,0.50,11.3,14.5,16,70,1.5
1,01-005,北海道稚内市,221,0.50,11.3,14.5,16,70,1.5
2,01-005,北海道稚内市,810,0.50,11.3,14.5,16,70,1.5
3,01-010,北海道浦河町,3800,0.36,3.7,3.5,29,60,0.3
4,01-015,北海道木古内町,2010,0.17,10,12.7,21,70,0.5


In [103]:
water_df.describe()
water_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3619 entries, 0 to 3618
Data columns (total 9 columns):
5位数字ID    3619 non-null object
地点或企业     3619 non-null object
水量        3591 non-null object
硝酸盐浓度     3619 non-null float64
Na浓度      3619 non-null object
Cl浓度      3619 non-null object
硬度        3619 non-null object
蒸发残留物     3619 non-null object
TOC       3619 non-null float64
dtypes: float64(2), object(7)
memory usage: 254.5+ KB


### 异常值处理

有217行测量结果不是数值型

In [104]:
detect_columns = ['水量', '硝酸盐浓度', 'Na浓度', 'Cl浓度', '硬度', '蒸发残留物', 'TOC',]
df_for_drop = water_df[~water_df[detect_columns].applymap(np.isreal).all(1)]
print(df_for_drop.shape)
df_for_drop.head(10)

(217, 9)


,5位数字ID,地点或企业,水量,硝酸盐浓度,Na浓度,Cl浓度,硬度,蒸发残留物,TOC
21,01-044,北海道足寄町,2100,0.20,<4.0,1.3,9,80,0.7
81,02-010,青森県深浦町,19,0.04,54,42,<1,230,0.3
109,02-052,青森県六ヶ所村,41,0.20,15.8,9.7,<34,153,0.4
131,02-171,青森県むつ市,(m3)休止中,0.35,15.6,28.2,44,141,0.3
211,03-012,岩手県陸前高田市,"2,908 (m3)休止中",0.85,11.8,18.1,79,139,0.6
212,03-012,岩手県陸前高田市,181(m3)休止中,0.70,6.6,5.6,63,93,0.5
215,03-014,岩手県紫波町,247,4.33,8.7,<10.3,78,<162,0.4
257,04-013,宮城県多賀城市,216(m3)休止中,0.05,178,32.6,10,471,1.3
274,04-074,宮城県加美町,3608,0.02,<10.6,<4.2,<28,142,0.3
275,04-074,宮城県加美町,397,0.02,<9.7,4.4,<24,<137,0.3


In [105]:
# df[df.地点或企业.isin(['北海道足寄町'])]
# df[df.地点或企业.isin(['三重県四日市市'])]
# df_for_drop.index

In [106]:
# 将 "<10.3" 的数值, 替换为 10.3 数值

def replace_lt(x):
  if isinstance(x, str) and x[0] == '<':
    return float(x[1:])
  elif isinstance(x, (float, int)):
    return x
  else:
    raise ValueError(f'cannot parse {x} {type(x)}')


water_df['硝酸盐浓度'] = water_df['硝酸盐浓度'].map(replace_lt)
water_df['Na浓度'] = water_df['Na浓度'].map(replace_lt)
water_df['Cl浓度'] = water_df['Cl浓度'].map(replace_lt)
water_df['硬度'] = water_df['硬度'].map(replace_lt)
water_df['蒸发残留物'] = water_df['蒸发残留物'].map(replace_lt)
water_df['TOC'] = water_df['TOC'].map(replace_lt)


# 将 "2,908 (m3)休止中" 的数值, 替换为 2908 数值

def replace_volume(x):
  if isinstance(x, str) and x.endswith('(m3)休止中'):
    x = x.split('(m3)')[0].strip()
    return int(x.replace(',', '')) if x else np.NaN
  elif isinstance(x, (float, int)):
    return x
  else:
    raise ValueError(f'cannot parse {x} {type(x)}')

water_df['水量'] = water_df['水量'].map(replace_volume)

然后又有47行没有水量, 

一部分来自原表空缺, 

一部分由于原表写的是 '(m3)休止中', 刚才也被替换了 NaN

In [107]:
print(water_df[water_df.isnull().any(1)].shape)
water_df[water_df.isnull().any(1)]


(47, 9)


,5位数字ID,地点或企业,水量,硝酸盐浓度,Na浓度,Cl浓度,硬度,蒸发残留物,TOC
131,02-171,青森県むつ市,NaN,0.35,15.6,28.2,44.0,141.0,0.3
364,06-041,山形県金山町,NaN,0.20,7.2,9.3,18.0,53.0,0.6
378,07-010,福島県須賀川市,NaN,0.05,10.5,2.2,26.0,139.0,0.3
431,07-067,福島県西郷村,NaN,0.92,7.6,4.9,95.0,170.0,0.5
681,10-001,群馬県高崎市,NaN,0.10,9.8,1.2,38.0,120.0,0.3
688,10-001,群馬県高崎市,NaN,0.91,8.7,2.2,56.0,150.0,0.3
800,10-053,群馬県渋川市,NaN,3.70,8.0,5.0,56.0,142.0,0.3
888,11-038,埼玉県上尾市,NaN,0.05,8.8,5.2,77.0,148.0,0.9
973,12-004,千葉県松戸市,NaN,1.89,16.0,24.1,69.0,160.0,0.8
1190,15-074,新潟県上越市,NaN,0.10,43.0,28.0,39.0,190.0,0.5


In [108]:
# 删去没有水量的数据
water_df.drop(index=water_df[water_df.isnull().any(1)].index, inplace=True)

In [109]:
# 这时所有的行都是有效数字了, 不再出现异常的行
detect_columns = ['水量', '硝酸盐浓度', 'Na浓度', 'Cl浓度', '硬度', '蒸发残留物', 'TOC',]
water_df[~water_df[detect_columns].applymap(np.isreal).all(1)]

,5位数字ID,地点或企业,水量,硝酸盐浓度,Na浓度,Cl浓度,硬度,蒸发残留物,TOC


In [110]:
# 总共 3572 行
water_df.shape

(3572, 9)

### 对硝酸盐浓度, Na浓度等, 以水量为权重做加权平均

In [111]:
# test

# df = (pd.DataFrame.from_dict(dict([('ID', [1, 1, 1, 2, 2]),
#                                   ('HeightA', [1, 2, 3, 4, 5]), 
#                                   ('WeightA', [10, 10, 10, 0, 3]),
#                                   ('HeightB', [2, 4, 6, 8, 10]), 
#                                   ('WeightB', [1, 1, 2, 1, 2])]),
#                             ))
# print (df)

# def givewm(weightcolumn):
#   def f1(x): return np.ma.average(x, weights=df.loc[x.index, weightcolumn])
#   return f1

# df2 = df.groupby('ID').agg({'HeightA': givewm('WeightA'),
#                             'HeightB': givewm('WeightB')
#                            })
# print (df2)

In [112]:
def givewm(weightcolumn):
  def f1(x): return np.ma.average(x, weights=water_df.loc[x.index, weightcolumn])
  return f1

water_df_agg = water_df.groupby('地点或企业').agg({
   '5位数字ID': 'first',
   '水量': 'sum',
   '硝酸盐浓度': givewm('水量'),
   'Na浓度': givewm('水量'),
   'Cl浓度': givewm('水量'),
   '硬度': givewm('水量'),
   '蒸发残留物': givewm('水量'),
   'TOC': givewm('水量'),
}).reset_index()

water_df_agg

,地点或企业,5位数字ID,水量,硝酸盐浓度,Na浓度,Cl浓度,硬度,蒸发残留物,TOC
0,三重県いなべ市,24-010,16339.0,0.903,5.131,4.089,87.891,120.088,0.313
1,三重県亀山市,24-015,19071.0,0.926,10.730,11.764,53.530,113.528,0.300
2,三重県伊勢市,24-009,31149.0,1.418,6.296,5.942,56.860,101.628,0.300
3,三重県伊賀市,24-005,13917.0,0.886,7.629,8.826,50.603,91.713,0.695
4,三重県四日市市,24-004,73166.0,1.747,9.423,8.506,62.611,114.655,0.263
5,三重県多気町,24-043,2066.0,2.181,6.274,6.598,64.169,108.914,0.300
6,三重県尾鷲市,24-014,9935.0,0.200,3.700,3.700,9.000,18.000,0.300
7,三重県御浜町,24-055,3770.0,0.754,5.800,6.122,23.699,72.398,0.100
8,三重県明和町,24-048,5478.0,2.492,8.038,7.781,50.781,100.095,0.300
9,三重県朝日町,24-021,2213.0,0.966,9.793,7.264,87.244,178.179,0.370


### 检出 "地点or企业" 中不是 "市/町" 的值



In [113]:
# 都道府县
todofuken_data = '''
中文	日文	假名	首府	地域	区域	人口[表 1]	面积[表 2]	人口密度[表 3]	郡	市町村	ISO 3166-2
爱知县	愛知県	あいちけん	名古屋市	中部	本州	7,532,231	5,153.90	1,438	10	63	JP-23
秋田县	秋田県	あきたけん	秋田市	东北	本州	1,029,196	11,612.11	94	6	25	JP-05
青森县	青森県	あおもりけん	青森市	东北	本州	1,323,861	9,606.24	142	8	40	JP-02
千叶县	千葉県	ちばけん	千叶市	关东	本州	6,283,602	5,156.15	1,204	6	56	JP-12
爱媛县	愛媛県	えひめけん	松山市	四国	四国	1,405,325	5,676.44	252	7	20	JP-38
福井县	福井県	ふくいけん	福井市	中部	本州	794,433	4,188.76	192	7	17	JP-18
福冈县	福岡県	ふくおかけん	福冈市	九州	九州	5,126,389	4,971.01	1,020	15	69	JP-40
福岛县	福島県	ふくしまけん	福岛市	东北	本州	1,938,559	13,782.54	147	13	61	JP-07
岐阜县	岐阜県	ぎふけん	岐阜市	中部	本州	2,066,266	10,598.18	196	9	42	JP-21
群马县	群馬県	ぐんまけん	前桥市	关东	本州	1,998,275	6,363.16	315	9	39	JP-10
广岛县	広島県	ひろしまけん	广岛市	中国	本州	2,857,475	8,476.95	337	6	23	JP-34
北海道	北海道	ほっかいどう	札幌市	北海道	北海道	5,370,807	83,453.57	66	64	180	JP-01
兵库县	兵庫県	ひょうごけん	神户市	近畿	本州	5,606,545	8,392.42	666	8	41	JP-28
茨城县	茨城県	いばらきけん	水户市	关东	本州	2,960,458	6,095.62	487	7	44	JP-08
石川县	石川県	いしかわけん	金泽市	中部	本州	1,153,627	4,185.32	279	6	19	JP-17
岩手县	岩手県	いわてけん	盛冈市	东北	本州	1,277,271	15,278.51	87	11	35	JP-03
香川县	香川県	かがわけん	高松市	四国	四国	997,811	1,861.70	535	5	17	JP-37
鹿儿岛县	鹿児島県	かごしまけん	鹿儿岛市	九州	九州	1,668,003	9,132.42	187	11	49	JP-46
神奈川县	神奈川県	かながわけん	横滨市	关东	本州	9,155,389	2,415.42	3,746	7	35	JP-14
高知县	高知県	こうちけん	高知市	四国	四国	732,535	7,104.70	108	6	35	JP-39
熊本县	熊本県	くまもとけん	熊本市	九州	九州	1,798,149	6,908.45	263	10	48	JP-43
京都府	京都府	きょうとふ	京都市	近畿	本州	2,569,410	4,612.93	571	6	28	JP-26
三重县	三重県	みえけん	津市	近畿	本州	1,841,753	5,760.72	322	7	29	JP-24
宫城县	宮城県	みやぎけん	仙台市	东北	本州	2,319,438	6,861.51	342	10	36	JP-04
宫崎县	宮崎県	みやざきけん	宫崎市	九州	九州	1,119,544	6,684.67	170	8	31	JP-45
长野县	長野県	ながのけん	长野市	中部	本州	2,126,064	12,598.48	171	14	81	JP-20
长崎县	長崎県	ながさきけん	长崎市	九州	九州	1,392,950	4,092.80	349	4	23	JP-42
奈良县	奈良県	ならけん	奈良市	近畿	本州	1,380,181	3,691.09	379	7	39	JP-29
新潟县	新潟県	にいがたけん	新潟市	中部	本州	2,300,923	12,582.37	189	9	35	JP-15
大分县	大分県	おおいたけん	大分市	九州	九州	1,176,891	5,804.24	206	3	18	JP-44
冈山县	岡山県	おかやまけん	冈山市	中国	本州	1,927,632	7,008.63	278	12	29	JP-33
冲绳县	沖縄県	おきなわけん	那霸市	冲绳	冲绳	1,467,071	2,271.57	613	5	41	JP-47
大阪府	大阪府	おおさかふ	大阪市	近畿	本州	8,861,437	1,893.18	4,683	5	43	JP-27
佐贺县	佐賀県	さがけん	佐贺市	九州	九州	837,977	2,439.23	348	7	23	JP-41
埼玉县	埼玉県	さいたまけん	埼玉市	关东	本州	7,343,807	3,767.09	1,909	9	71	JP-11
滋贺县	滋賀県	しがけん	大津市	近畿	本州	1,420,260	4,017.36	351	5	26	JP-25
岛根县	島根県	しまねけん	松江市	中国	本州	696,382	6,707.32	107	7	21	JP-32
静冈县	静岡県	しずおかけん	静冈市	中部	本州	3,756,865	7,328.61	513	9	42	JP-22
栃木县	栃木県	とちぎけん	宇都宫市	关东	本州	1,991,597	6,408.28	313	6	33	JP-09
德岛县	徳島県	とくしまけん	德岛市	四国	四国	764,213	4,145.26	189	8	24	JP-36
东京都	東京都	とうきょうと	新宿区	关东	本州	13,530,053	2,102.35	6,259	1	39	JP-13
鸟取县	鳥取県	とっとりけん	鸟取市	中国	本州	575,264	3,507.19	168	4	19	JP-31
富山县	富山県	とやまけん	富山市	中部	本州	1,074,705	4,247.22	257	2	15	JP-16
和歌山县	和歌山県	わかやまけん	和歌山市	近畿	本州	984,689	4,725.58	212	6	30	JP-30
山形县	山形県	やまがたけん	山形市	东北	本州	1,118,468	9,323.34	125	8	35	JP-06
山口县	山口県	やまぐちけん	山口市	中国	本州	1,408,588	6,110.76	238	5	22	JP-35
山梨县	山梨県	やまなしけん	甲府市	中部	本州	844,717	4,465.37	193	5	28	JP-19
'''.strip().split('\n')

todofuken_dict = {line.split('\t')[-1][3:]: line.split('\t')[1] for line in todofuken_data[1:]}

todofuken_dict

{'01': '北海道',
 '02': '青森県',
 '03': '岩手県',
 '04': '宮城県',
 '05': '秋田県',
 '06': '山形県',
 '07': '福島県',
 '08': '茨城県',
 '09': '栃木県',
 '10': '群馬県',
 '11': '埼玉県',
 '12': '千葉県',
 '13': '東京都',
 '14': '神奈川県',
 '15': '新潟県',
 '16': '富山県',
 '17': '石川県',
 '18': '福井県',
 '19': '山梨県',
 '20': '長野県',
 '21': '岐阜県',
 '22': '静岡県',
 '23': '愛知県',
 '24': '三重県',
 '25': '滋賀県',
 '26': '京都府',
 '27': '大阪府',
 '28': '兵庫県',
 '29': '奈良県',
 '30': '和歌山県',
 '31': '鳥取県',
 '32': '島根県',
 '33': '岡山県',
 '34': '広島県',
 '35': '山口県',
 '36': '徳島県',
 '37': '香川県',
 '38': '愛媛県',
 '39': '高知県',
 '40': '福岡県',
 '41': '佐賀県',
 '42': '長崎県',
 '43': '熊本県',
 '44': '大分県',
 '45': '宮崎県',
 '46': '鹿児島県',
 '47': '沖縄県'}

In [114]:
def to_todofuken(s):
  return todofuken_dict[s[:2]]

def to_shichoson(s, t):
  if t not in s:
    raise ValueError(f'not found {t} in {s}')
  return s.replace(t, '', 1) # 只替换一次

# 以 `5位数字ID` 转换出 `都道府县`
# 以 `地点或企业` 减去 `都道府县` 作为 `市町村或企业` 字段

water_df_agg['都道府县'] = water_df_agg['5位数字ID'].map(to_todofuken)
water_df_agg['市町村或企业'] = water_df_agg.apply(lambda row: to_shichoson(row['地点或企业'], row['都道府县']), axis=1)
water_df_agg

,地点或企业,5位数字ID,水量,硝酸盐浓度,Na浓度,Cl浓度,硬度,蒸发残留物,TOC,都道府县,市町村或企业
0,三重県いなべ市,24-010,16339.0,0.903,5.131,4.089,87.891,120.088,0.313,三重県,いなべ市
1,三重県亀山市,24-015,19071.0,0.926,10.730,11.764,53.530,113.528,0.300,三重県,亀山市
2,三重県伊勢市,24-009,31149.0,1.418,6.296,5.942,56.860,101.628,0.300,三重県,伊勢市
3,三重県伊賀市,24-005,13917.0,0.886,7.629,8.826,50.603,91.713,0.695,三重県,伊賀市
4,三重県四日市市,24-004,73166.0,1.747,9.423,8.506,62.611,114.655,0.263,三重県,四日市市
5,三重県多気町,24-043,2066.0,2.181,6.274,6.598,64.169,108.914,0.300,三重県,多気町
6,三重県尾鷲市,24-014,9935.0,0.200,3.700,3.700,9.000,18.000,0.300,三重県,尾鷲市
7,三重県御浜町,24-055,3770.0,0.754,5.800,6.122,23.699,72.398,0.100,三重県,御浜町
8,三重県明和町,24-048,5478.0,2.492,8.038,7.781,50.781,100.095,0.300,三重県,明和町
9,三重県朝日町,24-021,2213.0,0.966,9.793,7.264,87.244,178.179,0.370,三重県,朝日町


In [115]:
def is_shichoson(s):
  if '(' in s:
    s = s.split('(')[0]
  return s.endswith(('市', '町', '村', ))

# 找出 `市町村或企业` 不符合规范的, 通常是 水道企業団 等, 一共 49 行
print(water_df_agg[~water_df_agg['市町村或企业'].apply(is_shichoson)].shape)
water_df_agg[~water_df_agg['市町村或企业'].apply(is_shichoson)]

(49, 11)


,地点或企业,5位数字ID,水量,硝酸盐浓度,Na浓度,Cl浓度,硬度,蒸发残留物,TOC,都道府县,市町村或企业
61,兵庫県播磨高原広域事務組合,28-094,2102.0,2.300,13.000,35.000,76.000,190.000,0.100,兵庫県,播磨高原広域事務組合
65,兵庫県淡路広域水道企業団,28-097,19442.0,0.847,16.261,16.151,53.847,161.706,0.743,兵庫県,淡路広域水道企業団
70,兵庫県西播磨水道企業団,28-018,19444.0,1.007,9.178,8.910,44.073,111.653,0.283,兵庫県,西播磨水道企業団
117,北海道西空知広域水道企業団,01-117,2789.0,0.050,4.600,2.700,13.000,39.000,0.700,北海道,西空知広域水道企業団
121,千葉県三芳水道企業団,12-097,12384.0,2.246,32.019,41.747,89.532,224.569,0.800,千葉県,三芳水道企業団
123,千葉県八匝水道企業団,12-070,11132.0,3.360,30.600,50.500,93.000,238.000,1.100,千葉県,八匝水道企業団
128,千葉県千葉県,12-001,215650.0,1.605,18.216,23.756,78.821,168.199,0.828,千葉県,千葉県
139,千葉県山武郡市広域水道企業団,12-071,22517.0,3.035,28.484,46.588,89.526,232.366,1.200,千葉県,山武郡市広域水道企業団
158,千葉県長生郡市広域市町村圏組,12-072,54144.0,2.253,23.738,37.484,91.254,222.059,1.031,千葉県,長生郡市広域市町村圏組
186,埼玉県坂戸、鶴ヶ島水道企業団,11-062,10745.0,0.565,19.983,4.388,47.917,136.444,0.502,埼玉県,坂戸、鶴ヶ島水道企業団


这些 `水道企業団` 等, 需要转换为以地点代替, 下面从 enterprise 表中查找对应的地点

### 补充 Enterprise to location.xlsx

> 传了一个Enterprise to location，这里头可以把企业变成地点。  
> 它一个企业同时给好几个市供水，那就把这好几个地点的化学指标定为昨天算出来的该企业的数据。  
> 地点之间，我用空格做分词符。

这个 "Enterprise to location" 提供了 "自来水企业 -> 市" 的映射, 以及 "自来水企业 -> 净水厂" 的映射, 
都是一对多的映射

昨天的数据是 "市 -> 净水厂" 的一对多映射

这三者是什么关系? 
 
另外, 两份数据都有净水厂编号, 但是编号似乎对不上, 这编号没有意义?


> enterprise to location的一对多映射，跟昨天的市->净水厂的一对多映射，可能是互补关系。编号有意义，应该是primary key，在两张表里是一样的。上表可能包含下表，但是反过来不成立

> enterprise.col(A)等价water.col(A)  
> enterprise.col(C)等价water.col(B)  
> enterprise.col(B+D)等价water.col(B)当中的大部分地名

In [116]:
location_df = pd.read_excel("../Enterprise to location.xlsx")

In [117]:
location_df.head(5)

,事業主体ID,県,事業主体名,市町村,浄水場名,水源名,原水の種類,1日平均浄水量,Unnamed: 8,大腸菌(定量)(MPN/100ml),...,ジェオスミン,2-メチルイソボルネオール,非イオン界面活性剤,フェノール類,有機物(TOCの量),pH値,味,臭気,色度,濁度
0,28-094,兵庫,播磨高原広域事務組合,たつの市 上郡町 佐用町,01-00新宮新水源,新宮新水源(新宮新水源、曽我井水源と混,NaN,(m3)休止中,1,0.0,...,<0.000001,<0.000001,<0.005,<0.0005,0.1,6.3,0,0,<0.5,<0.1
1,28-094,兵庫,播磨高原広域事務組合,たつの市 上郡町 佐用町,03-00川向水源,川向水源,浅井戸水,2102,4,0.0,...,<0.000001,<0.000001,<0.005,<0.0005,0.1,6.5,0,0,<0.5,<0.1
2,23-034,愛知,海部南部水道企業団,愛西市 弥富市 飛島村 蟹江町,01-00立田配水場,県水受水(尾張西部),浄水受水,5406,0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,23-034,愛知,海部南部水道企業団,愛西市 弥富市 飛島村 蟹江町,02-00弥富配水場,県水受水(尾張西部),浄水受水,16122,0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,23-034,愛知,海部南部水道企業団,愛西市 弥富市 飛島村 蟹江町,03-00佐屋配水場,県水受水(尾張西部),浄水受水,8537,0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [118]:
# Enterprise to location.xlsx 保留企业名地名, 除掉该表中的检测数据

# location_df = location_df['事業主体ID	県	事業主体名	市町村	浄水場名'.split('\t')]
location_df = location_df['事業主体ID	県	事業主体名	市町村'.split('\t')]


explore_df(location_df)

==== row, col ==== (257, 4)
==== columns ====
事業主体ID	県	事業主体名	市町村
==== describe ====
        事業主体ID    県      事業主体名            市町村
count      257  257        257            257
unique      49   22         47             47
top     28-097   兵庫  淡路広域水道企業団  南あわじ市 洲本市 淡路市
freq        46   52         46             46
==== info ====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257 entries, 0 to 256
Data columns (total 4 columns):
事業主体ID    257 non-null object
県         257 non-null object
事業主体名     257 non-null object
市町村       257 non-null object
dtypes: object(4)
memory usage: 8.1+ KB
None
==== sample ====


,事業主体ID,県,事業主体名,市町村
221,45-029,宮崎,一ツ瀬川営農飲雑用水広域事務組合,西都市 新富町 高鍋町 木城町
97,20-037,長野,佐久水道企業団,東御市 佐久市 佐久穂町 御代田町
31,32-015,島根,斐川宍道水道企業団,出雲市 松江市
157,28-097,兵庫,淡路広域水道企業団,南あわじ市 洲本市 淡路市
41,43-013,熊本,大津菊陽水道企業団,大津町 菊陽町


> Enterprise to location可以把企业变成地点  
> 一个企业同时给好几个市供水，那就把这好几个地点的化学指标定为昨天算出来的该企业的数据  
> 地点之间用空格分词

In [119]:
# 检查 location_df 的 事業主体ID 和 water_df 的 ID

water_df_ids = set(water_df['5位数字ID'])
location_df_ids = set(location_df['事業主体ID'])

print('water_df 超出 location_df 的 ID 数量: ', len(water_df_ids - location_df_ids))
print('location_df 超出 water_df 的 ID 数量: ', len(location_df_ids - water_df_ids), '分别是: ',location_df_ids - water_df_ids)


water_df 超出 location_df 的 ID 数量:  940
location_df 超出 water_df 的 ID 数量:  17 分别是:  {'01-101', '41-004', '25-030', '35-020', '08-016', '02-046', '12-039', '01-142', '47-009', '43-029', '06-038', '25-016', '04-065', '01-126', '08-015', '40-096', '23-034'}


In [120]:
# location_df 里一个企业可能对应了多个水厂, 需要精简, 
# 事業主体ID 保留一个即可
location_df = location_df.groupby('事業主体ID').agg({
   '県': 'first',
   '事業主体名': 'first',
   '市町村': 'first',
}).reset_index()

location_df

,事業主体ID,県,事業主体名,市町村
0,01-101,北海道,長幌上水道企業団,長沼町 南幌町
1,01-117,北海道,西空知広域水道企業団,新十津川町 雨竜町 浦臼町
2,01-126,北海道,月新水道企業団,月形町
3,01-142,北海道,中空知広域水道企業団,滝川市 砂川市 歌志内市
4,02-046,青森,久吉ダム水道企業団,平川市 大鰐町
5,02-051,青森,八戸圏域水道企業団,八戸市 七戸町 六戸町 三戸町 五戸町 おいらせ町 南部町
6,02-053,青森,津軽広域水道企業団,弘前市 黒石市 五所川原市 つがる市 藤崎町 田舎館村 板柳町 鶴田町
7,04-065,宮城,石巻地方広域水道企業団,石巻市 東松島市
8,06-036,山形,尾花沢市大石田町環境衛,尾花沢市 大石田町
9,06-038,山形,最上川中部水道企業団,中山町 山辺町 山形市


### 将 water_df_agg 划分为两部分, 处理企业名的部分

In [137]:
# 检出 water_df_agg 中, `市町村或企业` 中不符合规范的行

water_df_agg_shichoson = water_df_agg[water_df_agg['市町村或企业'].apply(is_shichoson)]   # 符合规范的
water_df_agg_enterprise = water_df_agg[~water_df_agg['市町村或企业'].apply(is_shichoson)] # 不符合规范的, 49 rows


# 以下重点处理 water_df_agg_enterprise

# 连接 water_df_agg_enterprise + location_df 上, 通过企业ID
water_df_agg_enterprise = pd.merge(water_df_agg_enterprise, location_df, how='left', on=None, left_on='5位数字ID', right_on='事業主体ID')
water_df_agg_enterprise

,地点或企业,5位数字ID,水量,硝酸盐浓度,Na浓度,Cl浓度,硬度,蒸发残留物,TOC,都道府县,市町村或企业,事業主体ID,県,事業主体名,市町村
0,兵庫県播磨高原広域事務組合,28-094,2102.0,2.300,13.000,35.000,76.000,190.000,0.100,兵庫県,播磨高原広域事務組合,28-094,兵庫,播磨高原広域事務組合,たつの市 上郡町 佐用町
1,兵庫県淡路広域水道企業団,28-097,19442.0,0.847,16.261,16.151,53.847,161.706,0.743,兵庫県,淡路広域水道企業団,28-097,兵庫,淡路広域水道企業団,南あわじ市 洲本市 淡路市
2,兵庫県西播磨水道企業団,28-018,19444.0,1.007,9.178,8.910,44.073,111.653,0.283,兵庫県,西播磨水道企業団,28-018,兵庫,西播磨水道企業団,相生市 たつの市
3,北海道西空知広域水道企業団,01-117,2789.0,0.050,4.600,2.700,13.000,39.000,0.700,北海道,西空知広域水道企業団,01-117,北海道,西空知広域水道企業団,新十津川町 雨竜町 浦臼町
4,千葉県三芳水道企業団,12-097,12384.0,2.246,32.019,41.747,89.532,224.569,0.800,千葉県,三芳水道企業団,12-097,千葉,三芳水道企業団,館山市 南房総市
5,千葉県八匝水道企業団,12-070,11132.0,3.360,30.600,50.500,93.000,238.000,1.100,千葉県,八匝水道企業団,12-070,千葉,八匝水道企業団,横芝光町 匝瑳市
6,千葉県千葉県,12-001,215650.0,1.605,18.216,23.756,78.821,168.199,0.828,千葉県,千葉県,NaN,NaN,NaN,NaN
7,千葉県山武郡市広域水道企業団,12-071,22517.0,3.035,28.484,46.588,89.526,232.366,1.200,千葉県,山武郡市広域水道企業団,12-071,千葉,山武郡市広域水道企業団,東金市 大網白里市 九十九里町 山武市 横芝光町
8,千葉県長生郡市広域市町村圏組,12-072,54144.0,2.253,23.738,37.484,91.254,222.059,1.031,千葉県,長生郡市広域市町村圏組,12-072,千葉,長生郡市広域市町村圏組合,茂原市 一宮町 睦沢町 長生村 白子町 長柄町 長南町
9,埼玉県坂戸、鶴ヶ島水道企業団,11-062,10745.0,0.565,19.983,4.388,47.917,136.444,0.502,埼玉県,坂戸、鶴ヶ島水道企業団,11-062,埼玉,坂戸、鶴ヶ島水道企業団,坂戸市 鶴ヶ島市


这里看到许多企业都能配对, 但仍有一些 (通过ID) 找不到, 只能扔了

In [138]:
# 扔掉 NaN 的行
water_df_agg_enterprise.drop(water_df_agg_enterprise[water_df_agg_enterprise['市町村'].isna()].index, inplace=True)

# 扔掉 没用的 columns
water_df_agg_enterprise.drop(columns='事業主体ID 県 事業主体名'.split(' '), inplace=True)
water_df_agg_enterprise

,地点或企业,5位数字ID,水量,硝酸盐浓度,Na浓度,Cl浓度,硬度,蒸发残留物,TOC,都道府县,市町村或企业,市町村
0,兵庫県播磨高原広域事務組合,28-094,2102.0,2.300,13.000,35.000,76.000,190.000,0.100,兵庫県,播磨高原広域事務組合,たつの市 上郡町 佐用町
1,兵庫県淡路広域水道企業団,28-097,19442.0,0.847,16.261,16.151,53.847,161.706,0.743,兵庫県,淡路広域水道企業団,南あわじ市 洲本市 淡路市
2,兵庫県西播磨水道企業団,28-018,19444.0,1.007,9.178,8.910,44.073,111.653,0.283,兵庫県,西播磨水道企業団,相生市 たつの市
3,北海道西空知広域水道企業団,01-117,2789.0,0.050,4.600,2.700,13.000,39.000,0.700,北海道,西空知広域水道企業団,新十津川町 雨竜町 浦臼町
4,千葉県三芳水道企業団,12-097,12384.0,2.246,32.019,41.747,89.532,224.569,0.800,千葉県,三芳水道企業団,館山市 南房総市
5,千葉県八匝水道企業団,12-070,11132.0,3.360,30.600,50.500,93.000,238.000,1.100,千葉県,八匝水道企業団,横芝光町 匝瑳市
7,千葉県山武郡市広域水道企業団,12-071,22517.0,3.035,28.484,46.588,89.526,232.366,1.200,千葉県,山武郡市広域水道企業団,東金市 大網白里市 九十九里町 山武市 横芝光町
8,千葉県長生郡市広域市町村圏組,12-072,54144.0,2.253,23.738,37.484,91.254,222.059,1.031,千葉県,長生郡市広域市町村圏組,茂原市 一宮町 睦沢町 長生村 白子町 長柄町 長南町
9,埼玉県坂戸、鶴ヶ島水道企業団,11-062,10745.0,0.565,19.983,4.388,47.917,136.444,0.502,埼玉県,坂戸、鶴ヶ島水道企業団,坂戸市 鶴ヶ島市
10,埼玉県桶川北本水道企業団,11-048,34875.0,0.053,34.125,7.830,58.120,196.056,1.936,埼玉県,桶川北本水道企業団,桶川市 北本市


In [139]:
# 按 `市町村` 的空格拆开 

water_df_agg_enterprise = water_df_agg_enterprise.join(water_df_agg_enterprise['市町村'].str.split(' ', expand=True) \
                                                       .stack() \
                                                       .reset_index(level=1, drop=True) \
                                                       .rename('市町村split')
                                                      )
water_df_agg_enterprise.head(20)

# 这里拆开后, `水量` 就被重复了多次, 未来加权时有问题

,地点或企业,5位数字ID,水量,硝酸盐浓度,Na浓度,Cl浓度,硬度,蒸发残留物,TOC,都道府县,市町村或企业,市町村,市町村split
0,兵庫県播磨高原広域事務組合,28-094,2102.0,2.300,13.000,35.000,76.000,190.000,0.100,兵庫県,播磨高原広域事務組合,たつの市 上郡町 佐用町,たつの市
0,兵庫県播磨高原広域事務組合,28-094,2102.0,2.300,13.000,35.000,76.000,190.000,0.100,兵庫県,播磨高原広域事務組合,たつの市 上郡町 佐用町,上郡町
0,兵庫県播磨高原広域事務組合,28-094,2102.0,2.300,13.000,35.000,76.000,190.000,0.100,兵庫県,播磨高原広域事務組合,たつの市 上郡町 佐用町,佐用町
1,兵庫県淡路広域水道企業団,28-097,19442.0,0.847,16.261,16.151,53.847,161.706,0.743,兵庫県,淡路広域水道企業団,南あわじ市 洲本市 淡路市,南あわじ市
1,兵庫県淡路広域水道企業団,28-097,19442.0,0.847,16.261,16.151,53.847,161.706,0.743,兵庫県,淡路広域水道企業団,南あわじ市 洲本市 淡路市,洲本市
1,兵庫県淡路広域水道企業団,28-097,19442.0,0.847,16.261,16.151,53.847,161.706,0.743,兵庫県,淡路広域水道企業団,南あわじ市 洲本市 淡路市,淡路市
2,兵庫県西播磨水道企業団,28-018,19444.0,1.007,9.178,8.910,44.073,111.653,0.283,兵庫県,西播磨水道企業団,相生市 たつの市,相生市
2,兵庫県西播磨水道企業団,28-018,19444.0,1.007,9.178,8.910,44.073,111.653,0.283,兵庫県,西播磨水道企業団,相生市 たつの市,たつの市
3,北海道西空知広域水道企業団,01-117,2789.0,0.050,4.600,2.700,13.000,39.000,0.700,北海道,西空知広域水道企業団,新十津川町 雨竜町 浦臼町,新十津川町
3,北海道西空知広域水道企業団,01-117,2789.0,0.050,4.600,2.700,13.000,39.000,0.700,北海道,西空知広域水道企業団,新十津川町 雨竜町 浦臼町,雨竜町


In [140]:
# 重命名 `市町村split` -> `市町村或企业`
water_df_agg_enterprise['市町村或企业'] = water_df_agg_enterprise['市町村split'] 
# 去掉多余字段
water_df_agg_enterprise.drop(columns=['市町村', '市町村split'], inplace=True)
water_df_agg_enterprise

,地点或企业,5位数字ID,水量,硝酸盐浓度,Na浓度,Cl浓度,硬度,蒸发残留物,TOC,都道府县,市町村或企业
0,兵庫県播磨高原広域事務組合,28-094,2102.0,2.300,13.000,35.000,76.000,190.000,0.100,兵庫県,たつの市
0,兵庫県播磨高原広域事務組合,28-094,2102.0,2.300,13.000,35.000,76.000,190.000,0.100,兵庫県,上郡町
0,兵庫県播磨高原広域事務組合,28-094,2102.0,2.300,13.000,35.000,76.000,190.000,0.100,兵庫県,佐用町
1,兵庫県淡路広域水道企業団,28-097,19442.0,0.847,16.261,16.151,53.847,161.706,0.743,兵庫県,南あわじ市
1,兵庫県淡路広域水道企業団,28-097,19442.0,0.847,16.261,16.151,53.847,161.706,0.743,兵庫県,洲本市
1,兵庫県淡路広域水道企業団,28-097,19442.0,0.847,16.261,16.151,53.847,161.706,0.743,兵庫県,淡路市
2,兵庫県西播磨水道企業団,28-018,19444.0,1.007,9.178,8.910,44.073,111.653,0.283,兵庫県,相生市
2,兵庫県西播磨水道企業団,28-018,19444.0,1.007,9.178,8.910,44.073,111.653,0.283,兵庫県,たつの市
3,北海道西空知広域水道企業団,01-117,2789.0,0.050,4.600,2.700,13.000,39.000,0.700,北海道,新十津川町
3,北海道西空知広域水道企業団,01-117,2789.0,0.050,4.600,2.700,13.000,39.000,0.700,北海道,雨竜町


In [148]:
# 拼接 water_df_agg_shichoson + water_df_agg_enterprise
water_df_agg_concat = pd.concat([water_df_agg_shichoson, water_df_agg_enterprise])

# 调整字段顺序
columns = ['5位数字ID', '地点或企业', '都道府县', '市町村或企业', '水量', '硝酸盐浓度', 'Na浓度', 'Cl浓度', '硬度', '蒸发残留物', 'TOC', ] 
water_df_agg_concat = water_df_agg_concat[columns]

explore_df(water_df_agg_concat)

==== row, col ==== (1020, 11)
==== columns ====
5位数字ID	地点或企业	都道府县	市町村或企业	水量	硝酸盐浓度	Na浓度	Cl浓度	硬度	蒸发残留物	TOC
==== describe ====
               水量     硝酸盐浓度      Na浓度      Cl浓度        硬度     蒸发残留物       TOC
count    1020.000  1020.000  1020.000  1020.000  1020.000  1020.000  1020.000
mean    11352.904     1.207    13.226    12.811    60.764   135.473     0.453
std     19081.719     1.289    12.714    20.131    48.138    74.751     0.434
min        11.000     0.010     0.000     0.000     0.000     0.000     0.100
25%      2291.000     0.380     6.596     4.900    37.000    86.000     0.300
50%      5542.000     0.800     9.852     8.205    52.038   125.417     0.300
75%     12885.500     1.561    15.992    14.336    73.368   168.318     0.487
max    267213.000    11.100   197.255   501.144   910.000  1308.605     6.261
==== info ====
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1020 entries, 0 to 47
Data columns (total 11 columns):
5位数字ID    1020 non-null object
地点或企业     1020 non-null

,5位数字ID,地点或企业,都道府县,市町村或企业,水量,硝酸盐浓度,Na浓度,Cl浓度,硬度,蒸发残留物,TOC
48,28-045,兵庫県丹波市(中央),兵庫県,丹波市(中央),16903.0,0.683,17.903,29.602,57.337,141.723,0.722
142,12-014,千葉県我孫子市,千葉県,我孫子市,8448.0,0.020,43.000,36.200,62.000,252.000,0.800
118,01-138,北海道訓子府町,北海道,訓子府町,673.0,1.702,6.919,3.215,40.529,120.461,0.690
104,01-071,北海道洞爺湖町,北海道,洞爺湖町,4517.0,0.807,18.094,21.296,151.191,273.015,0.279
455,38-005,愛媛県今治市(今治),愛媛県,今治市(今治),12602.0,1.371,6.801,4.701,48.601,95.023,0.530


In [143]:
# 随便选一个市, 这里的数据来自 location_merged_df + water_df_agg, 以 企业ID 连接
#find_in_df(location_merged_df, col='市町村split', pat='山武市')

In [142]:
# 同样这个市, 这里的数据来自 water_df_agg 中每个市的水量加权平均
#find_in_df(water_df_agg, col='地点或企业', pat='山武市')

# ???

In [149]:
print(water_df_agg_concat.to_csv(), file=open('市町村水质结果.csv', 'w'))